# Import Library for Keras_ocr servers np

In [1]:
from flask import Flask, request, jsonify
import keras_ocr
import cv2, os
import numpy as np
import base64
import tensorflow as tf
from keras import backend as K

# Load the pre-trained model

In [2]:
pipeline = keras_ocr.pipeline.Pipeline()

Looking for C:\Users\User\.keras-ocr\craft_mlt_25k.h5
Looking for C:\Users\User\.keras-ocr\crnn_kurapan.h5


# Function image_decoder for predict from the images

In [3]:
def predict_ocr(file):
    image = cv2.imdecode(np.frombuffer(file.read(), np.uint8), cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #predict from the image
    predictions = pipeline.recognize([image])
    #print(predictions)
    text = ' '.join([word[0] for word in predictions[0]])
    #Construct the text from json 
    json_text = {}
    #put bounding box to every texts
    for i in range(len(predictions[0])):
        json_text[f"param{i+1}"] = predictions[0][i][0]
        bounding_boxes = predictions[0][i][1]
        pt1 = tuple([int(bounding_boxes[0][0]),int(bounding_boxes[0][1])])
        pt2 = tuple([int(bounding_boxes[2][0]),int(bounding_boxes[2][1])])
        cv2.rectangle(image, pt1, pt2, (0, 255, 0), 2)
        cv2.putText(image, predictions[0][i][0], pt1, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1, cv2.LINE_AA)
        # Encode the image as base64
        _, image_encoded = cv2.imencode('.png', image)
        image_base64 = base64.b64encode(image_encoded).decode('utf-8')
        # Return the recognized text and image with bounding boxes as a JSON response
        response = {'json': json_text, 'image_base64': image_base64}
    return response

# Create Flask Server

In [ ]:
app = Flask(__name__)
@app.route('/ocr', methods=['POST'])
def ocr():
    # Get the image from the request and decode
    file = request.files['image']
    response = predict_ocr(file)
    return jsonify(response)
    
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.33:5000
Press CTRL+C to quit
